# Cross Cluster Replication APIs
In this notebook, we will go over the xCluster APIs in YBA with examples and the steps required to make those API calls. We assume that you have already two universes deployed in your YBA and their universe uuids are known.
## Get the API Token 
All xCluster replication YBA APIs are restricted to only authenticated users, so to use them through API, you first need to get the API token using the following code:

In [43]:
import requests
import os
from pprint import pprint

yba_url = os.getenv("YBA_URL", "http://192.168.56.102:9000")
yba_user = {
    "email": os.getenv("YBA_USER_EMAIL", "admin"),
    "password": os.getenv("YBA_USER_PASSWORD", "admin"),
}

route = f"{yba_url}/api/v1/api_login"
payload = {
    "email": yba_user["email"],
    "password": yba_user["password"],
}
response = requests.post(url=route, json=payload).json()
pprint(response)

customer_uuid = response["customerUUID"]
yba_api_token = response["apiToken"]
headers = {"X-AUTH-YW-API-TOKEN": yba_api_token}

{'apiToken': '2894987a-4fdf-47cf-95a2-be3f2039cf79',
 'apiTokenVersion': 0,
 'customerUUID': 'f33e3c9b-75ab-4c30-80ad-cba85646ea39',
 'userUUID': 'a9190fe6-1067-409f-95cf-dbd96893c9c9'}


Then you can use `customer_uuid` as a url parameter and pass the `yba_api_token` in the request header with name `X-AUTH-YW-API-TOKEN` to show that the user is authenticated.

## Get Storage Config UUID
XCluster uses backup/restore for replicating existing data on the source universe, and a storage config is required to store the backup and then restore from. To get the storage config uuid, use the following code:

In [44]:
route = f"{yba_url}/api/v1/customers/{customer_uuid}/configs"
response = requests.get(url=route, headers=headers).json()
storage_configs = list(filter(lambda config: config["type"] == "STORAGE", response))
if len(storage_configs) < 1:
    print("No storage config found")
    exit(-1)

storage_config_uuid = storage_configs[0]["configUUID"]
print(storage_config_uuid)

301029d7-ebcc-4cd9-9dbb-c87775340d09


## List Tables on Source Universe
XCluster APIs are at the table granularity. You need to pass the list of tables you would like to replicate to the xCluster API request bodies.
Please note that although the xCluster APIs are at table granularity, but YBA only supports bootstrapping of YSQL tables with DB granularity, so if you would like to do an xCluster operation that requires bootstrapping of YSQL tables, you need to pass all the table UUIDs in a database.
The following is how you can get the list of the tables for a universe:

In [45]:
source_universe_uuid = os.getenv(
    "YBA_SOURCE_UNIVERSE_UUID", "1e4f0c62-7c7a-40b8-bc3c-0a5d0e2a99c3"
)
route = (f"{yba_url}/api/v1/customers/{customer_uuid}/universes/{source_universe_uuid}/tables"
         f"?includeParentTableInfo={str(False).lower()}&onlySupportedForXCluster={str(True).lower()}")
response = requests.get(url=route, headers=headers).json()
all_ysql_tables_uuid_list = [
    table["tableUUID"]
    for table in list(
        filter(lambda table: table["tableType"] == "PGSQL_TABLE_TYPE", response)
    )
]
pprint(all_ysql_tables_uuid_list)

['0000407b-0000-3000-8000-000000004086',
 '0000407b-0000-3000-8000-000000004081',
 '0000407b-0000-3000-8000-00000000407c',
 '000033f3-0000-3000-8000-00000000407a',
 '000033f3-0000-3000-8000-000000004075',
 '000033f3-0000-3000-8000-000000004070']


## Waiting For XCluster Tasks
The xCluster APIs will create a task in the backend and returns a task uuid which you can follow to see the progress and the status of the task. You can use the following function to wait for a task:

In [46]:
import time

def waitForTask(task_uuid):
    route = f"{yba_url}/api/v1/customers/{customer_uuid}/tasks/{task_uuid}"
    while True:
        response = requests.get(url=route, headers=headers).json()
        status = response["status"]
        if status == "Failure":
            route = f"{yba_url}/api/customers/{customer_uuid}/tasks/{task_uuid}/failed"
            response = requests.get(url=route, headers=headers)
            if response is not None:
                response = response.json()
                if "failedSubTasks" in response:
                    errors = [
                        subtask["errorString"] for subtask in response["failedSubTasks"]
                    ]
                    print(f"Task {task_uuid} failed with the following errors:")
                    print("\n".join(errors))
                else:
                    pprint(response)
            else:
                print(
                    f"Task {task_uuid} failed, but could not get the failure messages"
                )
            exit(-1)
        elif status == "Success":
            print(f"Task {task_uuid} finished successfully")
            break
        print(f"Waiting for task {task_uuid}...")
        time.sleep(20)

## Creating XCluster Configs
Now we have all the required information to successfully create a cross cluster replication from universe `source_universe_uuid` to `target_universe_uuid`. `configType` can be either `Basic` or `Txn`. `Txn` provides transactional guarantees while replicating the data.
Please note that unless you have good reasons to skip bootstrapping, the list of tables in `.tables` and `.bootstrapParams.tables` should be the same.

In [47]:
target_universe_uuid = os.getenv(
    "YBA_TARGET_UNIVERSE_UUID", "574d4b62-4940-4184-ad9f-f1b99c7bc495"
)
route = f"{yba_url}/api/v1/customers/{customer_uuid}/xcluster_configs"
payload = {
    "sourceUniverseUUID": source_universe_uuid,
    "targetUniverseUUID": target_universe_uuid,
    "name": "my-xcluster-config",
    "tables": all_ysql_tables_uuid_list,
    "configType": "Basic",  # It could be Basic or Txn.
    "bootstrapParams": {  # You can omit this field to forcefully avoid bootstrapping.
        "tables": all_ysql_tables_uuid_list,
        "backupRequestParams": {"storageConfigUUID": storage_config_uuid},
    },
}
response = requests.post(url=route, json=payload, headers=headers).json()
pprint(response)
if "taskUUID" not in response:
    print(f"Failed to create the task: {response}")
    exit(-1)

waitForTask(response["taskUUID"])

xcluster_config_uuid = response["resourceUUID"]

{'resourceUUID': '769b74d6-80b4-465e-852e-ab93b8e2c4fc',
 'taskUUID': '91834a5b-db5c-4a50-952a-2b506f2f1e26'}
Waiting for task 91834a5b-db5c-4a50-952a-2b506f2f1e26...
Task 91834a5b-db5c-4a50-952a-2b506f2f1e26 finished successfully


## Getting XCluster Configs
You can get the xCluster config using its uuid. See the following example.

In [48]:
route = f"{yba_url}/api/v1/customers/{customer_uuid}/xcluster_configs/{xcluster_config_uuid}"
response = requests.get(url=route, headers=headers).json()
pprint(response)

{'createTime': '2023-11-28T17:37:21Z',
 'imported': False,
 'lag': {'tserver_async_replication_lag_micros': {'data': [],
                                                  'directURLs': ['http://localhost:9090/graph?g0.expr=avg%28max+by+%28exported_instance%2C+saved_name%29%28%7Bnode_prefix%3D%22yb-admin-hzare-gcp-1-src%22%2C+saved_name%3D%7E%22async_replication_sent_lag_micros%7Casync_replication_committed_lag_micros%22%2C+stream_id%3D%7E%228e6c53f4157f4efcbf9057976a736c28%7C7d5d5da42ceb48d4b2cf30af4ac3b79e%7Ce4d4dfaac5164bcbae7b24df7350437e%7Cd1fa5c98b3354ef6840ba365e72dc4a9%7Ce0f835e3dc5c465b8097b4650280ecb3%7C6f44a6dd14b84476bd8f2edb517e4990%22%7D%29%29+by+%28exported_instance%2C+saved_name%29+%2F+1000&g0.tab=0&g0.range_input=3600s&g0.end_input='],
                                                  'layout': {'title': 'Async '
                                                                      'Replication '
                                                                      'Lag

## Modifying Tables in XCluster Configs
You can add/remove tables to/from an existing xCluster config. This is useful when you would like to add new tables to your database after the replication is set up, or you would like to drop a table from your database.
Please note that to drop a table from your database, *first you need to remove that table from the xCluster config.*
To modify the tables in replication in an xCluster config, you need to pass the list of the tables that you would like to be in replication. In other words, you remove the table UUIDs that you do not want to be replicated, and add the new table uuids you want to replication. See the following example. 

In [49]:
# Remove tables.
route = f"{yba_url}/api/v1/customers/{customer_uuid}/xcluster_configs/{xcluster_config_uuid}"
payload = {
    "tables": all_ysql_tables_uuid_list[:-2],
}
response = requests.put(url=route, json=payload, headers=headers).json()
pprint(response)
if "taskUUID" not in response:
    print(f"Failed to create the task: {response}")
    exit(-1)

waitForTask(response["taskUUID"])

# Add tables.
route = f"{yba_url}/api/v1/customers/{customer_uuid}/xcluster_configs/{xcluster_config_uuid}"
payload = {
    "tables": all_ysql_tables_uuid_list,
    "bootstrapParams": {  # You can omit this field to forcefully avoid bootstrapping.
        "tables": all_ysql_tables_uuid_list,
        "backupRequestParams": {"storageConfigUUID": storage_config_uuid},
    },
}
response = requests.put(url=route, json=payload, headers=headers).json()
pprint(response)
if "taskUUID" not in response:
    print(f"Failed to create the task: {response}")
    exit(-1)

waitForTask(response["taskUUID"])

{'resourceUUID': '769b74d6-80b4-465e-852e-ab93b8e2c4fc',
 'taskUUID': 'ddb8633b-1337-40fa-971c-8bc64f8fb19d'}
Waiting for task ddb8633b-1337-40fa-971c-8bc64f8fb19d...
Task ddb8633b-1337-40fa-971c-8bc64f8fb19d finished successfully
{'resourceUUID': '769b74d6-80b4-465e-852e-ab93b8e2c4fc',
 'taskUUID': '608b6c71-7945-41f7-8ac0-b17530645f70'}
Waiting for task 608b6c71-7945-41f7-8ac0-b17530645f70...
Task 608b6c71-7945-41f7-8ac0-b17530645f70 finished successfully


## Reconciling XCluster Configs with YBDB State
Sometimes, it is required to make changes to the replication group using yb-admin. In these cases, the corresponding xCluster config in YBA will not be automatically updated to learn about the yb-admin changes, and a manual synchronization call is required as follows:

In [50]:
route = (
    f"{yba_url}/api/v1/customers/{customer_uuid}/xcluster_configs/sync"
    f"?targetUniverseUUID={target_universe_uuid}"
)
response = requests.post(url=route, headers=headers).json()
if "taskUUID" not in response:
    print(f"Failed to create the task: {response}")
    exit(-1)

waitForTask(response["taskUUID"])

Waiting for task 9cc7ff3d-d0fa-4c8a-aad5-f10bd29e29d3...
Task 9cc7ff3d-d0fa-4c8a-aad5-f10bd29e29d3 finished successfully


## Restarting XCluster Configs
The replication between two universes can break for various reasons including temporary network partitions. In these cases, after the issue is resolved, you can restart replication. You may also include index tables to the replication by restarting the replication for their main tables.

In [51]:
route = f"{yba_url}/api/v1/customers/{customer_uuid}/xcluster_configs/{xcluster_config_uuid}"
payload = {
    "tables": all_ysql_tables_uuid_list,
    "bootstrapParams": {
        "backupRequestParams": {"storageConfigUUID": storage_config_uuid},
    },
}
response = requests.post(url=route, json=payload, headers=headers).json()
pprint(response)
if "taskUUID" not in response:
    print(f"Failed to create the task: {response}")
    exit(-1)

waitForTask(response["taskUUID"])

{'resourceUUID': '769b74d6-80b4-465e-852e-ab93b8e2c4fc',
 'taskUUID': '9309cb89-6ab1-4d74-9390-fdf1d7c5a4df'}
Waiting for task 9309cb89-6ab1-4d74-9390-fdf1d7c5a4df...
Task 9309cb89-6ab1-4d74-9390-fdf1d7c5a4df finished successfully


## Pausing/Resuming XCluster Configs
You can pause the replication for some time and then resume it. Please note that if the replication is paused for an extended period of time, a replication restart is required.

In [52]:
# Pause the replication.
route = f"{yba_url}/api/v1/customers/{customer_uuid}/xcluster_configs/{xcluster_config_uuid}"
payload = {"status": "Paused"}
response = requests.put(url=route, json=payload, headers=headers).json()
pprint(response)
if "taskUUID" not in response:
    print(f"Failed to create the task: {response}")
    exit(-1)

waitForTask(response["taskUUID"])

# Resume the replication.
route = f"{yba_url}/api/v1/customers/{customer_uuid}/xcluster_configs/{xcluster_config_uuid}"
payload = {"status": "Running"}
response = requests.put(url=route, json=payload, headers=headers).json()
pprint(response)
if "taskUUID" not in response:
    print(f"Failed to create the task: {response}")
    exit(-1)

waitForTask(response["taskUUID"])

{'resourceUUID': '769b74d6-80b4-465e-852e-ab93b8e2c4fc',
 'taskUUID': 'ad277f56-2bb8-4d35-9037-63adc03daa73'}
Waiting for task ad277f56-2bb8-4d35-9037-63adc03daa73...
Task ad277f56-2bb8-4d35-9037-63adc03daa73 finished successfully
{'resourceUUID': '769b74d6-80b4-465e-852e-ab93b8e2c4fc',
 'taskUUID': 'c90287e0-e5fe-462d-9330-752ab4e08d18'}
Waiting for task c90287e0-e5fe-462d-9330-752ab4e08d18...
Task c90287e0-e5fe-462d-9330-752ab4e08d18 finished successfully


## Deleting XCluster Configs
You can delete the xCluster config so there is no replication relation between the two universes. Please note that `isForceDelete` is useful when one of the universes is not available or there is an issue with the config. In those cases, you pass `True` and it will ignore errors and delete the config.

In [53]:
route = (f"{yba_url}/api/v1/customers/{customer_uuid}/xcluster_configs/{xcluster_config_uuid}"
         f"?isForceDelete={str(False).lower()}")
response = requests.delete(url=route, headers=headers).json()
pprint(response)
if "taskUUID" not in response:
    print(f"Failed to create the task: {response}")
    exit(-1)

waitForTask(response["taskUUID"])

{'resourceUUID': '769b74d6-80b4-465e-852e-ab93b8e2c4fc',
 'taskUUID': '04e7370b-d00e-44f1-8393-f1f21f2a9f1f'}
Waiting for task 04e7370b-d00e-44f1-8393-f1f21f2a9f1f...
Task 04e7370b-d00e-44f1-8393-f1f21f2a9f1f finished successfully
